In [ ]:
class Scraper():
    results = []
    
    def getResults(self):
        return self.results
    
    def Run(self, search):
        raise Exception("Base Class Run Called")

In [ ]:
import requests
from faker import Faker
fake = Faker()
from bs4 import BeautifulSoup

In [ ]:
class WebScraper(Scraper):
    session = None
    
    def getClient(self):
        if self.session is None:
            self.session = requests.Session()
            self.session.headers['user-agent'] = fake.chrome()
        return self.session
    
    def getSearchUrl(self, search, page):
        return f"https://testing.to/search/{search}/{page}/"
    
    def Run(self, search):
        client = self.getClient()
        url = self.getSearchUrl(search)
        res = client.get(url)
        soup =BeautifulSoup(res.text)

In [ ]:
class WebScraper1337x(WebScraper):
    
    def getSearchUrl(self, search, page):
        return f"https://1337x.to/sort-search/{search}/seeders/desc/{page}/"
    
    def Run(self, search,pages = 3):
        client = self.getClient()
        titles = ['name', 'seed', 'leech', 'time', 'size', 'uploader']
        for page_number in range(1,pages+1):
            url = self.getSearchUrl(search,page_number)
            res = client.get(url)
            soup =BeautifulSoup(res.text)
            table = soup.find_all('table')[0]
            for row in table.find_all('tr'):
                tds = [i.text.strip() for i in row.find_all('td')]
                out = dict(zip(titles,tds))
                if len(out.keys())>0:
                    yield out

In [ ]:
for result in WebScraper1337x().Run("Sing 2"):
    print(result['name'])

In [ ]:
class WebScraperkickassTorrents_to(WebScraper):
    
    def getSearchUrl(self, search, page):
        return f"https://kickasstorrents.to/usearch/{search}/{page}/?sortby=seeders&sort=desc"
    
    def Run(self, search,pages = 1):
        client = self.getClient()
        titles = ['name', 'size', 'uploader', 'time', 'seed', 'leech']
        for page_number in range(1,pages+1):
            url = self.getSearchUrl(search,page_number)
            res = client.get(url)
            soup =BeautifulSoup(res.text)
            table = soup.find_all('table')[0]
            table = table.find_all('table')[0]
            table = table.find_all('table')[0]
            for row in table.find_all('tr'):
                tds = [i.text.strip() for i in row.find_all('td')]
                out = dict(zip(titles,tds))
                if len(out.keys())>0:
                    out['name'] = out['name'].split('\n')[0]
                    yield out

In [ ]:
for result in WebScraperkickassTorrents_to().Run("Sing 2"):
    print(result['name'])

In [ ]:

class WebScraperRarbg(WebScraper):
    
    def getSearchUrl(self, search, page):
        return f"https://rarbg.to/torrents.php?search={search}&order=seeders&by=DESC&page={page}"
    
    def Run(self, search,pages = 1):
        client = self.getClient()
        titles = ['cat', 'name', 'time', 'size', 'seed', 'leech','comments','uploader']
        for page_number in range(1,pages+1):
            url = self.getSearchUrl(search,page_number)
            res = client.get(url)
            soup =BeautifulSoup(res.text)
            table = soup.find('table',{'class':'lista2t'})
            for row in table.find_all('tr'):
                tds = [i.text.strip() for i in row.find_all('td')]
                out = dict(zip(titles,tds))
                if len(out.keys())>0:
                    if out['cat'] =='Cat.':
                        continue
                    del out['cat']
                    yield out

In [ ]:
for result in WebScraperRarbg().Run("Sing 2"):
    print(result['name'])